In [1]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
import sklearn
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [4]:
df_anime = pd.read_csv("../data/df_anime_export.csv")
df_profile = pd.read_csv("../data/df_profile_export.csv")
df_review = pd.read_csv("../data/df_review_export.csv")
df_an_pro_rev = pd.read_csv("../data/merged_rev_df_export.csv")
df_an_pro = pd.read_csv("../data/merged_df_export.csv")
df_main = pd.read_csv("../data/df_prof_rev_anime_clean.csv")

In [5]:
df_main

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,profile,review_score,review_score_indepth,text_preprocessed,Unnamed: 0.1,gender,birthday,favorites_anime
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
1,8,32935,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,"After the victory against Aoba Jousai High, Ka...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 8, 2016 to Dec 10, 2016",10.0,395375,223,16.0,8.94,"victory aoba jousai high , karasuno high schoo...",skrn,8,"{'Overall': '8', 'Story': '9', 'Animation': '9...","oh boy , finally stepped game good-but-not-gre...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
2,16,30276,One Punch Man,The seemingly ordinary and unimpressive Saitam...,"['Action', 'Sci-Fi', 'Comedy', 'Parody', 'Supe...","Oct 5, 2015 to Dec 21, 2015",12.0,1475219,5,50.0,8.68,seemingly ordinary unimpressive saitama rather...,skrn,7,"{'Overall': '7', 'Story': '6', 'Animation': '1...",one punch man reminds successful hollywood act...,2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
3,24,4107,Tengen Toppa Gurren Lagann Movie 1: Gurren-hen,Animation studio Gainax presented a website fo...,"['Action', 'Comedy', 'Mecha']","Sep 6, 2008",1.0,112211,1003,349.0,8.18,animation studio gainax presented website rele...,skrn,7,"{'Overall': '7', 'Story': '6', 'Animation': '9...","movie recap first half tv series , % r use mat...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
4,32,4081,Natsume Yuujinchou,"While most fifteen-year-old boys, in one way o...","['Slice of Life', 'Demons', 'Supernatural', 'D...","Jul 8, 2008 to Sep 30, 2008",13.0,344207,276,176.0,8.37,"fifteen-year-old boy , one way another , harbo...",skrn,4,"{'Overall': '4', 'Story': '3', 'Animation': '5...",friend strongly recommended series insisted ne...,2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130514,479833,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",sensei69,5,"{'Overall': '5', 'Story': '5', 'Animation': '9...",* * * going spoiler free review ... * * * okay...,77347,Male,"Jul 16, 1999","['16774', '23605', '30', '5114', '9253', '437'..."
130515,479834,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ssjtk,8,"{'Overall': '8', 'Story': '8', 'Animation': '1...","n't listen critic , usually idea talking . mov...",77348,NaN,"Jul 14, 1996","['6045', '15583', '6547', '14289', '1535', '31..."
130516,479835,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",Ground_zero,9,"{'Overall': '9', 'Story': '7', 'Animation': '1...",final fantasy fan since old day although 's fa...,77349,Male,"Apr 7, 1982","['43', '5114', '11061', '1', '543', '33082']"
130517,479836,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ClawViper,7,"{'Overall': '7', 'Story': '6', 'Animation': '1...",final fantasy fan following development final ...,77350,NaN,Jul 19,"['13125', '38524', '14397']"


## Collab Filter a bit cleaner

In [15]:
df_touse = df_main[["profile", "uid", "review_score"]]

In [23]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ..."


In [24]:
df_anime_touse = df_anime[["uid", "title"]]

In [16]:
df = Dataset.load_from_df(df_touse, Reader(rating_scale=(1,10)))
trainset = df.build_full_trainset()

In [17]:
model = SVD()
# model.fit(trainset)
cross_validate(model, df, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9684  1.9695  1.9932  1.9831  1.9851  1.9799  0.0095  
MAE (testset)     1.5138  1.5244  1.5369  1.5245  1.5338  1.5267  0.0081  
Fit time          5.33    5.40    5.40    5.35    5.36    5.37    0.03    
Test time         0.29    0.29    0.29    0.28    0.17    0.26    0.05    


{'test_rmse': array([1.96842287, 1.96951145, 1.9931785 , 1.98308366, 1.98508652]),
 'test_mae': array([1.51378406, 1.52444403, 1.53688054, 1.52445349, 1.53379802]),
 'fit_time': (5.3344175815582275,
  5.404222726821899,
  5.398222208023071,
  5.346611261367798,
  5.36325216293335),
 'test_time': (0.2890646457672119,
  0.29106616973876953,
  0.2890663146972656,
  0.2780625820159912,
  0.17103838920593262)}

In [22]:
model.predict('skrn', 30)

Prediction(uid='skrn', iid=30, r_ui=None, est=7.4955375382836325, details={'was_impossible': False})

In [25]:
user_id = 'skrn'
all_anime = df_touse['uid'].unique()
watched = df_touse[df_touse['profile']==user_id].uid
not_watched = [anime for anime in all_anime if anime not in watched]

# predict
score = [model.predict(user_id, anime_id) for anime_id in not_watched]
anime_id = []
pred_score = []
for i in range(0, len(score)):
    anime_id.append(score[i].iid)
    pred_score.append(score[i].est)
df_pred = pd.DataFrame({'uid':anime_id, 'pred_score':pred_score})

df_pred_real = df_pred.sort_values('pred_score', ascending=False).head(10)

df_pred_real.merge(df_anime_touse, how='left', on='uid')

,uid,pred_score,title
0,4181,9.208702,Clannad: After Story
1,5114,9.203944,Fullmetal Alchemist: Brotherhood
2,263,8.910416,Hajime no Ippo
3,28977,8.856539,Gintama°
4,9253,8.838419,Steins;Gate
5,1210,8.799281,NHK ni Youkoso!
6,2904,8.797443,Code Geass: Hangyaku no Lelouch R2
7,7311,8.793326,Suzumiya Haruhi no Shoushitsu
8,30654,8.764269,Ansatsu Kyoushitsu 2nd Season
9,37510,8.719953,Mob Psycho 100 II
